# Model Deployment

In [ ]:
!pip install gradio

Import the necessary library

In [ ]:
!pip install interpret

In [3]:
import gradio as gr
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np

In [5]:
from google.colab import files
upload=files.upload()

Saving daily crime numbers and weather data for time series analysis 1.csv to daily crime numbers and weather data for time series analysis 1.csv


The idea was to have multiple model be deployed and also displayed the visualization page. In Gradio we can have multiple pages for our machine learning model. Currently I created 2 page, it can be added more.

First page is Visualization page (Remarks: This is just a very basic visualization, we would need help on this to create meaningful visualization)

In [12]:
title = "Crime Rate Prediction Based on Weather Forecast"

#app 1 Visualisation
def visualization(name):
    #df = pd.read_csv('daily crime numbers and weather data for time series analysis.csv', index_col='date', parse_dates=['date'])
    df = pd.read_csv('daily crime numbers and weather data for time series analysis 1.csv', index_col='date', parse_dates=['date'])

    fig, ax = plt.subplots(figsize=(15, 4))
    df.plot(y='Offense Count', ax=ax, label='Dataset', title='Daily Crime Rate - Time Series Analysis', linewidth=1)
    plt.xlabel("Date")
    plt.ylabel('Offense Count')
    plt.grid()
    return fig

output = gr.Plot()


Second page is the prediction page, we are using EBM model as of now. There are total of 47 features according to the notebook. However, I have manually reset the value to zero if the feature is missing.

In [14]:
#app 2 Prediction
with open("ebm.pkl", "rb") as model_file:
    model = pickle.load(model_file)


def predict_crime_rate(month, year, is_holiday, is_weekend, tempmax, tempmin,
                       temp, feelslikemax, feelslikemin, feelslike, dew, humidity,
                       precip, precipcover, windgust, windspeed, winddir,
                       sealevelpressure, cloudcover, visibility, solarradiation,
                       uvindex, moonphase, Clear, Overcast, Partiallycloudy, Rain,
                       Snow, daytime_interval, lag_1_Offense_Count,
                       lag_2_Offense_Count, lag_3_Offense_Count, lag_4_Offense_Count,
                       lag_5_Offense_Count, lag_6_Offense_Count, lag_7_Offense_Count,
                       week_Friday, week_Monday, week_Saturday, week_Sunday,
                       week_Thursday, week_Tuesday, week_Wednesday, season_Autumn,
                       season_Spring, season_Summer, season_Winter):
    # Convert all inputs to a NumPy array
    input_data = np.array([
        [month, year, is_holiday, is_weekend, tempmax, tempmin, temp, feelslikemax, feelslikemin, feelslike, dew, humidity,
         precip, precipcover, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, uvindex,
         moonphase, Clear, Overcast, Partiallycloudy, Rain, Snow, daytime_interval, lag_1_Offense_Count,
         lag_2_Offense_Count, lag_3_Offense_Count, lag_4_Offense_Count, lag_5_Offense_Count, lag_6_Offense_Count, lag_7_Offense_Count,
         week_Friday, week_Monday, week_Saturday, week_Sunday, week_Thursday, week_Tuesday, week_Wednesday, season_Autumn,
         season_Spring, season_Summer, season_Winter]
    ])

    # Fill in missing values (None) with 0
    input_data = np.nan_to_num(input_data)

    # Make predictions using your model
    prediction = model.predict(input_data)
    return f"Predicted Crime Rate: {prediction[0]:.2f}"



This is our input, I believe we need 47 inputs here to have a better prediction. As of now this is the base of it, you can try to create more.

In [15]:
input_components = [
    gr.Dropdown([1,2,3,4,5,6,7,8,9,10,11,12], label="Month"),
    gr.Number(value=2023, label="Year"),
    gr.Checkbox(label="Is Holiday"),
    gr.Checkbox(label="Is Weekend"),
    gr.Slider(minimum=0, maximum=100, value=50, label="Temp Max (°C)"),
    gr.Slider(minimum=0, maximum=100, value=50, label="Temp Min (°C)"),
    gr.Slider(minimum=0, maximum=100, value=50, label="Temp (°C)"),
    gr.Slider(minimum=0, maximum=100, value=50, label="Feels Like Max (°C)"),
    gr.Slider(minimum=0, maximum=100, value=50, label="Feels Like Min (°C)"),
]

Putting everything together, this is how you launch the web application. Again, this is running on local, but i believe deployment of the web should be come next once we have a better idea on our web design.

In [16]:
#interface 1
app1 =  gr.Interface(fn = visualization, inputs=None, outputs=output)

#interface 2
app2 = gr.Interface(
    fn=predict_crime_rate,
    inputs=input_components,
    outputs="text",
    live=True,
    title="Crime Rate Prediction App",
    description="Enter the features for prediction. Missing values will be filled with 0.",
)

demo = gr.TabbedInterface([app1, app2], ["Home Page", "EBM Prediction"])


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:812: UserWarning: Expected 1 arguments for function <function visualization at 0x78c61d255f30>, received 0.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:816: UserWarning: Expected at least 1 arguments for function <function visualization at 0x78c61d255f30>, received 0.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:812: UserWarning: Expected 47 arguments for function <function predict_crime_rate at 0x78c61e1e9ea0>, received 9.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:816: UserWarning: Expected at least 47 arguments for function <function predict_crime_rate at 0x78c61e1e9ea0>, received 9.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1025: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


In [17]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c9fbb5c6407b7aa1fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
